In [1]:
# importing the necessary torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# defining the neural network class
class Net(nn.Module):
    
    # defining the constructor of the class
    def __init__(self):
        # invoke the constructor of the superclass
        super(Net, self).__init__()
        
        # 1 input image channel, 6 output channels, 5 x 5 square convolution
        
        # first convolutional layer
        # 1 input channel, 6 filters of dimension 5 x 5 are applied
        self.conv1 = nn.Conv2d(1, 6, 5)
        
        # second convolutional layer
        # 6 input channels, 16 filters of dimension 5 x 5 are applied
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # define the fully connected layers
        # affine operation: Wx + b = y
        
        # first fully connected layer
        # weight matrix of dimensions (16 * 5 * 5) = 400 x 120
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        
        # second fully connected layer
        # weight matrix of dimensions 120 x 84
        self.fc2 = nn.Linear(120, 84)
        
        # third fully connected layer
        # weight matrix of dimensions 84 x 10
        self.fc3 = nn.Linear(84, 10)
    
    # implement the forward pass
    def forward(self, x):
        # implement the first convolution
        x = self.conv1(x)
        
        # apply the ReLU activation
        x = F.relu(x)
        
        # apply max-pooling with a 2 x 2 filter
        x = F.max_pool2d(x, (2,2))
        
        # implement the second convolution
        x = self.conv2(x)
        
        # apply the ReLU activation
        x = F.relu(x)
        
        # apply max-pooling with a 2 x 2 filter
        x = F.max_pool2d(x, 2)
        
        # flatten the matrices across multiple channels into row vectors
        x = x.view(-1, self.num_flat_features(x))
        
        # implement forward pass through the first fully connected layer
        x = self.fc1(x)
        
        # implement the ReLU activation
        x = F.relu(x)

        # implement forward pass through the second fully connected layer
        x = self.fc2(x)
        
        # implement the ReLU activation
        x = F.relu(x)
        
        # implement forward pass through the third fully connected layer
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        # retrieve all dimensions except the batch dimension
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) 

10
torch.Size([6, 1, 5, 5])


In [5]:
# trying with a random 32 x 32 input
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0773,  0.1561, -0.0570,  0.0241,  0.0569, -0.0472,  0.0796, -0.0741,
         -0.0441,  0.1172]], grad_fn=<ThAddmmBackward>)


In [6]:
# set the gradient buffers of all parameters to 0 and backprop with random gradients
net.zero_grad()
out.backward(torch.randn(1, 10))

In [7]:
# calculating the loss
output = net(input)

# a dummy target, for example
target = torch.randn(10)  

# make it the same shape as output
target = target.view(1, -1)  

# set the loss function: Mean Squared Error
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.6555, grad_fn=<MseLossBackward>)


In [13]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [14]:
# implementing back propagation

# set the gradient buffers of all parameters to 0
net.zero_grad()     

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0318, -0.0042, -0.0058,  0.0231,  0.0036,  0.0177])


In [15]:
# update the parameters in the network
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in the training loop:

# set the gradient buffers to 0
optimizer.zero_grad()  

# implement the forward pass through the network
output = net(input)

# calculate the loss
loss = criterion(output, target)

# implement back propagation
loss.backward()

# update the weights in the network
optimizer.step()